In [1]:
import os
import re
import json5
import json
import math
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
from datasets import load_dataset, Dataset
from collections import Counter
from collections import Counter
from joblib import Parallel,delayed

In [2]:
origin_xlsx = '/mnt/users/LLMa/ChatGLM-6B/data/LUNG/origin_chest_ct_report_pid_reformat.csv'
percentile = 0.15


## Get report to conclusion

In [5]:
origin_df = pd.read_csv(origin_xlsx)

In [6]:
print(len(origin_df))
origin_df['report_evidences'].replace('', np.nan, inplace=True)
origin_df.dropna(subset=['report_evidences'], inplace=True)

origin_df['report_conclusion'].replace('', np.nan, inplace=True)
origin_df.dropna(subset=['report_conclusion'], inplace=True)

origin_df['report_conclusion'].replace('id:0', np.nan, inplace=True)
origin_df.dropna(subset=['report_conclusion'], inplace=True)
print(len(origin_df))


544570
529963


In [7]:
origin_df.replace('\s+', '', regex=True, inplace=True)

In [8]:
pid_set = list(set(list(origin_df['pid'])))
train_pids = pid_set[int(len(pid_set)*percentile):]
val_pids = pid_set[:int(len(pid_set)*percentile)]
print(len(pid_set), len(train_pids), len(val_pids))


392498 333624 58874


In [9]:
counter = Counter(list(origin_df['pid']))
his_pids = []
for pid,c in counter.items():
    if c>1:
        his_pids.append(pid)

In [247]:
def add_date_to_report(rows_df):
    reg = r'(((20)[1-2][0-9])((0[1-9]|1[0-2]))((0[1-9]|1[0-9])|2[0-9]|3[0-1]))'
    reg2 = r'(\.|\/|-|年)(([1-9](\.|\/|-)[1-9])|(\d{2}(\.|\/|-|月)[1-9])|([1-9](\.|\/|-)\d{2})|(\d{2}(\.|\/|-|月)\d{2}))(\D)'
    reg3 = r'(((20)[1-2][0-9])-((0[1-9]|1[0-2]))-((0[1-9]|1[0-9])|2[0-9]|3[0-1]))'
    date_strs=['']*len(rows_df)
    i=0
    for idx, row in rows_df.iterrows():
        # get current date
        study_instance_uid = row['study_instance_uid']
        compose_str = ''.join(study_instance_uid.split('.'))
        r1 = None
        r1= re.finditer(reg, study_instance_uid)
        for r in r1:
            date = r.group()
#             print(date)
            try:
                date = '{}{:02d}{}'.format(date[:4],int(date[4:-2]),str(date[-2:]))
                date =datetime.strptime(date, '%Y-%m-%d')
                date_str = '报告时间: {}。'.format(date)
                new_report = date_str+row['report_evidences']
                rows_df.at[idx, 'report_evidences'] = new_report
                rows_df.at[idx, 'date'] =str(date)
            except:
                pass
        if len(list(r1))==0:
            for t in study_instance_uid.split('.'):
                if len(t)==10:
                    loc_t = time.localtime(int(float(t)))
                    dt = time.strftime("%Y-%m-%d",loc_t)
                    date_str = '报告时间: {}。'.format(dt)
                    new_report = date_str+row['report_evidences']
                    rows_df.at[idx, 'report_evidences'] = new_report
                    rows_df.at[idx, 'date'] = dt
        
        # get previous date
        conculsion = row['report_conclusion']
        
        r2 = None
        r2= re.finditer(reg2, conculsion)
        for r_item in r2:
            date = r_item.group()[:-1]
            date_split = re.split('\.|-|\/|年|月',date)
            for i,item in enumerate(date_split):
                if len(item)==0:
                    continue
                try:
                    item = '{:02d}'.format(int(float(item)))
                except:
                    print(date)
                date_split[i]=item
            new_date = '-'.join(date_split)
            conculsion = conculsion.replace(date,new_date)
            rows_df.at[idx,'report_conclusion']=conculsion
        r3 = None
        r3 = re.search(reg3,conculsion)
        if r3 is not None:
            s,e = r3.span()
            pre_date = conculsion[s:e]
            rows_df.at[idx,'pervious_date']=pre_date
    return rows_df

In [248]:
origin_df_with_date = add_date_to_report(origin_df.copy())

In [262]:
def process_single(pid,origin_df_with_date):
    sel_df = origin_df_with_date.query('pid=="{}"'.format(pid))
    if len(sel_df)>1:
        sel_df=sel_df.sort_values(by='date')
    line = 0
    train_pair = []
    for idx,row in sel_df.iterrows():
        pre_date=row['pervious_date']
        date = str(row['date'])
        evidence = row['report_evidences']
        conlusion = row['report_conclusion']
        
        pre_row = sel_df.query('date=="{}"'.format(pre_date))
        history = ''
        if len(pre_row)!=0:
            history = list(pre_row['report_evidences'])[0]
        train_pair.append([pid,evidence,conlusion,history,date,pre_date])
    return train_pair

In [269]:
train_res = []
for pid in tqdm(his_pids):
    train_res+= process_single(pid,origin_df_with_date)

 19%|██████████████▊                                                               | 15903/83496 [05:51<25:31, 44.14it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 77%|████████████████████████████████████████████████████████████▏                 | 64423/83496 [23:03<06:41, 47.48it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [270]:
train_df = pd.DataFrame(data=train_res,columns=['pid','Input','Output','History','Date','Pre_Date'])
train_data = Dataset.from_pandas(train_df)
train_data.to_json('/mnt/users/LLMa/ChatGLM-6B/data/LUNG/train_conclusion_history.json',force_ascii=False)

Creating json from Arrow format:   0%|          | 0/221 [00:00<?, ?ba/s]

142981614

In [271]:
val_res = []
for pid in tqdm(val_pids):
    val_res+= proces_single(pid,origin_df_with_date)

100%|██████████████████████████████████████████████████████████████████████████████| 58874/58874 [16:55<00:00, 57.99it/s]


In [128]:
val_df = pd.DataFrame(data=val_res,columns=['Input','Output','History'])
val_data = Dataset.from_pandas(val_df)
val_data.to_json('/mnt/users/LLMa/ChatGLM-6B/data/LUNG/val_conclusion_history.json',force_ascii=False)